In [0]:
# MAGIC %md
# MAGIC # 🎓 06_Model_Training
# MAGIC Entrena un modelo de detección de fraude usando las features de la capa Features.
# MAGIC - Lee desde `fraude_qr.features.qr_tx_features_v1`
# MAGIC - Usa GroupKFold por `merchant_id` para evitar data leakage.
# MAGIC - Entrena con LogisticRegression, RandomForest y LightGBM (si disponible).
# MAGIC - Loggea métricas, parámetros y modelo en MLflow.
# MAGIC - Promueve el mejor modelo al Model Registry.

# COMMAND ----------

import mlflow
import mlflow.sklearn
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import joblib
import os

# --- 1. Configuración ---
features_table = "fraude_qr.features.qr_tx_features_v1"
experiment_name = "/Users/gastrofoodiebar@gmail.com/fraude-qr-experiment"
model_name = "fraude_qr_fraud_detection_model"

mlflow.set_experiment(experiment_name)

print(f"📖 Leyendo features desde: {features_table}")
print(f"🧪 Experimento MLflow: {experiment_name}")
print(f"📦 Nombre del modelo: {model_name}")

# --- 2. Cargar Datos de Features ---
df = spark.table(features_table).toPandas()
print(f"📊 Total registros: {len(df):,}")
print(f"📈 Tasa de fraude: {df['is_fraud'].mean():.4f}")

# --- 3. Preparar X, y, groups ---
feature_columns = [
    "amount",
    "distance_km",
    "payer_tx_count_1h",
    "payer_tx_count_24h",
    "amount_zscore_payer_7d"
]

X = df[feature_columns]
y = df["is_fraud"]
groups = df["merchant_id"]  # Para GroupKFold

print(f"📋 Features usadas: {feature_columns}")

# --- 4. Definir Modelos ---
models = {
    "LogisticRegression": LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
}

# --- 5. Entrenamiento con GroupKFold y MLflow ---
print("\n🚀 Iniciando entrenamiento con GroupKFold...")

gkf = GroupKFold(n_splits=5)

for model_name_key, model in models.items():
    print(f"\n--- Entrenando {model_name_key} ---")
    
    pr_auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Crear pipeline
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        
        # Entrenar
        pipeline.fit(X_train, y_train)
        
        # Predecir probabilidades
        y_proba = pipeline.predict_proba(X_val)[:, 1]
        
        # Calcular PR AUC
        pr_auc = average_precision_score(y_val, y_proba)
        pr_auc_scores.append(pr_auc)
        
        print(f"   Fold {fold + 1}: PR AUC = {pr_auc:.4f}")
    
    mean_pr_auc = np.mean(pr_auc_scores)
    std_pr_auc = np.std(pr_auc_scores)
    
    print(f"✅ {model_name_key} - PR AUC Promedio: {mean_pr_auc:.4f} ± {std_pr_auc:.4f}")
    
    # --- 6. Loggear en MLflow ---
    with mlflow.start_run(run_name=model_name_key):
        mlflow.log_param("model_type", model_name_key)
        mlflow.log_param("n_splits", 5)
        mlflow.log_metric("mean_pr_auc", mean_pr_auc)
        mlflow.log_metric("std_pr_auc", std_pr_auc)
        
        # Guardar pipeline como artefacto
        pipeline_filename = f"pipeline_{model_name_key}.pkl"
        joblib.dump(pipeline, pipeline_filename)
        mlflow.log_artifact(pipeline_filename)
        
        # Guardar thresholds
        precision, recall, thresholds = precision_recall_curve(y_val, y_proba)
        optimal_idx = np.argmax(precision >= 0.90)  # Umbral para precisión mínima del 90%
        optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5
        
        thresholds_dict = {"optimal_threshold": optimal_threshold}
        thresholds_filename = f"thresholds_{model_name_key}.json"
        pd.Series(thresholds_dict).to_json(thresholds_filename)
        mlflow.log_artifact(thresholds_filename)
        
        # Loggear modelo
        mlflow.sklearn.log_model(pipeline, "model")
        
        print(f"📝 Modelo y artefactos loggeados en MLflow para {model_name_key}.")

# --- 7. Promover el Mejor Modelo al Model Registry ---
print("\n🏆 Promoviendo el mejor modelo al Model Registry...")

# Encontrar el run con el mejor PR AUC
best_run = mlflow.search_runs(
    experiment_names=[experiment_name],
    order_by=["metrics.mean_pr_auc DESC"],
    max_results=1
).iloc[0]

run_id = best_run.run_id
model_uri = f"runs:/{run_id}/model"

print(f"🔝 Mejor modelo: run_id={run_id}, PR AUC={best_run['metrics.mean_pr_auc']:.4f}")

# Registrar modelo
mlflow.register_model(model_uri, model_name)

print(f"✅ Modelo registrado en Model Registry como '{model_name}'.")

# --- 8. Verificación ---
print("\n🔍 Verificando modelo en Model Registry:")
client = mlflow.tracking.MlflowClient()
latest_version = client.get_latest_versions(model_name, stages=["None"])[0]
print(f"   Versión: {latest_version.version}")
print(f"   Stage: {latest_version.current_stage}")
print(f"   Run ID: {latest_version.run_id}")

print("\n🎉 ¡Entrenamiento completado exitosamente!")

2025/09/20 01:07:47 INFO mlflow.tracking.fluent: Experiment with name '/Users/gastrofoodiebar@gmail.com/fraude-qr-experiment' does not exist. Creating a new experiment.


📖 Leyendo features desde: fraude_qr.features.qr_tx_features_v1
🧪 Experimento MLflow: /Users/gastrofoodiebar@gmail.com/fraude-qr-experiment
📦 Nombre del modelo: fraude_qr_fraud_detection_model
📊 Total registros: 7,362,604
📈 Tasa de fraude: 0.0014
📋 Features usadas: ['amount', 'distance_km', 'payer_tx_count_1h', 'payer_tx_count_24h', 'amount_zscore_payer_7d']

🚀 Iniciando entrenamiento con GroupKFold...

--- Entrenando LogisticRegression ---
   Fold 1: PR AUC = 0.0190
   Fold 2: PR AUC = 0.0213
   Fold 3: PR AUC = 0.0239
   Fold 4: PR AUC = 0.0198
   Fold 5: PR AUC = 0.0189
✅ LogisticRegression - PR AUC Promedio: 0.0206 ± 0.0019


2025/09/20 01:08:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


📝 Modelo y artefactos loggeados en MLflow para LogisticRegression.

--- Entrenando RandomForest ---
   Fold 1: PR AUC = 0.1030
   Fold 2: PR AUC = 0.1057
   Fold 3: PR AUC = 0.1065
   Fold 4: PR AUC = 0.0979
   Fold 5: PR AUC = 0.0974
✅ RandomForest - PR AUC Promedio: 0.1021 ± 0.0038


2025/09/20 04:26:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


📝 Modelo y artefactos loggeados en MLflow para RandomForest.

🏆 Promoviendo el mejor modelo al Model Registry...
🔝 Mejor modelo: run_id=5f6293705fb94e07841d77cd19803c6a, PR AUC=0.1021


Successfully registered model 'workspace.default.fraude_qr_fraud_detection_model'.


---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-8341375209016310>, line 142
    139 print(f"🔝 Mejor modelo: run_id={run_id}, PR AUC={best_run['metrics.mean_pr_auc']:.4f}")
    141 # Registrar modelo
--> 142 mlflow.register_model(model_uri, model_name)
    144 print(f"✅ Modelo registrado en Model Registry como '{model_name}'.")
    146 # --- 8. Verificación ---

File /databricks/python/lib/python3.12/site-packages/mlflow/tracking/_model_registry/fluent.py:80, in register_model(model_uri, name, await_registration_for, tags)
     20 def register_model(
     21     model_uri,
     22     name,
   (...)
     25     tags: Optional[dict[str, Any]] = None,
     26 ) -> ModelVersion:
     27     """Create a new model version in model registry for the model files specified by ``model_uri``.
     28 
     29     Note that this method assumes the model registry backend URI is the s

In [0]:
import mlflow

# --- 1. Define los parámetros del mejor modelo ---
# El ID del run que tuvo el mejor rendimiento (PR AUC = 0.1021)
best_run_id = "5f6293705fb94e07841d77cd19803c6a"

# El nombre que queremos darle al modelo en Unity Catalog (catalogo.esquema.nombre_modelo)
# Usaremos el catálogo y esquema que hemos definido para el proyecto.
model_name_uc = "fraude_qr.ml.detection_model_v1"

# --- 2. Construye la ruta al artefacto del modelo ---
# Esta es la dirección URI que apunta al modelo guardado dentro del run.
model_uri = f"runs:/{best_run_id}/model"

print(f"🏆 Promoviendo el modelo desde el run: {best_run_id}")
print(f"🔖 Registrando como: {model_name_uc}")

# --- 3. Registra el modelo ---
# Esta función crea una nueva versión del modelo en el Model Registry.
mlflow.register_model(
    model_uri=model_uri,
    name=model_name_uc
)

print(f"\n✅ ¡Modelo '{model_name_uc}' registrado exitosamente en Unity Catalog!")

🏆 Promoviendo el modelo desde el run: 5f6293705fb94e07841d77cd19803c6a
🔖 Registrando como: fraude_qr.ml.detection_model_v1


Successfully registered model 'fraude_qr.ml.detection_model_v1'.


---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-8341375209016317>, line 20
     16 print(f"🔖 Registrando como: {model_name_uc}")
     18 # --- 3. Registra el modelo ---
     19 # Esta función crea una nueva versión del modelo en el Model Registry.
---> 20 mlflow.register_model(
     21     model_uri=model_uri,
     22     name=model_name_uc
     23 )
     25 print(f"\n✅ ¡Modelo '{model_name_uc}' registrado exitosamente en Unity Catalog!")

File /databricks/python/lib/python3.12/site-packages/mlflow/tracking/_model_registry/fluent.py:80, in register_model(model_uri, name, await_registration_for, tags)
     20 def register_model(
     21     model_uri,
     22     name,
   (...)
     25     tags: Optional[dict[str, Any]] = None,
     26 ) -> ModelVersion:
     27     """Create a new model version in model registry for the model files specified by ``model_uri``.
     28 
 

In [0]:
import mlflow
from mlflow.models import infer_signature

# --- 1. Define los parámetros de tu mejor modelo ---
best_run_id = "5f6293705fb94e07841d77cd19803c6a"
model_name_uc = "fraude_qr.ml.detection_model_v1"
# Le daremos un nombre nuevo a la carpeta del artefacto para no sobreescribir el original
new_artifact_path = "model_with_signature" 

# --- 2. Carga el modelo que ya fue entrenado (el que no tiene firma) ---
print(f"Cargando el modelo original desde el run: {best_run_id}")
original_model_uri = f"runs:/{best_run_id}/model"
loaded_model = mlflow.sklearn.load_model(original_model_uri)

# --- 3. Crea la Firma ---
print("Creando la firma del modelo...")
input_example = X.head(5)
output_example = loaded_model.predict_proba(input_example)
signature = infer_signature(input_example, output_example)
print("✅ Firma creada.")

# --- 4. Reabre el Run y guarda una NUEVA versión del modelo CON la firma ---
print(f"Reabriendo el run '{best_run_id}' para añadir el nuevo artefacto del modelo...")
with mlflow.start_run(run_id=best_run_id):
    mlflow.sklearn.log_model(
        sk_model=loaded_model,
        artifact_path=new_artifact_path, # Lo guardamos en una nueva carpeta dentro del mismo run
        signature=signature,
        input_example=input_example
    )
print("✅ Nuevo artefacto del modelo con firma guardado en el run.")

# --- 5. Registra el NUEVO modelo (el que SÍ tiene firma) ---
print(f"\nAhora sí, registrando el modelo con firma como '{model_name_uc}'...")
new_model_uri = f"runs:/{best_run_id}/{new_artifact_path}"
mlflow.register_model(
    model_uri=new_model_uri,
    name=model_name_uc
)

print(f"\n🎉 ¡Modelo '{model_name_uc}' registrado exitosamente en Unity Catalog!")

Cargando el modelo original desde el run: 5f6293705fb94e07841d77cd19803c6a
Creando la firma del modelo...
✅ Firma creada.
Reabriendo el run '5f6293705fb94e07841d77cd19803c6a' para añadir el nuevo artefacto del modelo...


/databricks/python/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


✅ Nuevo artefacto del modelo con firma guardado en el run.

Ahora sí, registrando el modelo con firma como 'fraude_qr.ml.detection_model_v1'...


Registered model 'fraude_qr.ml.detection_model_v1' already exists. Creating a new version of this model...



🎉 ¡Modelo 'fraude_qr.ml.detection_model_v1' registrado exitosamente en Unity Catalog!


Created version '1' of model 'fraude_qr.ml.detection_model_v1'.
